# Human vs. supervised machine learning: Who learns patterns faster?

This program at hand contains all steps to conduct the experiment with machines in the paper "Human vs. supervised machine learning: Who learns patterns faster?"

Part 1: Preparation

Part 2: Matrices

Part 3: Rule

Part 4: Training and Test Set

Part 5: Grid Search (optional)

Part 6: Step-by-step Predicting (Optional)

Part 7: Conduct experiment with machines

Part 8: Conduct experiment with machines - CNN (Optional)

Part 9: Confusion matrix (Optional)

Part 10: Save Results

# Part 1: Preparation

In [ ]:
# import necessary libraries
import numpy as np
import pandas as pd

import itertools as it
import matplotlib.pyplot as plt
#conda install -c conda-forge imbalanced-learn
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix

from datetime import datetime

#pip install --upgrade tensorflow
from tensorflow.keras import datasets, layers, Sequential

# remove warnings
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=ConvergenceWarning) 



# Part 2: Matrices

In [ ]:
# Generate all binary 3x3 matrices (2^9 = 512)
# In Paper: 3. Methodology - 3.1.2. Implementation of task characteristics

listInteger  = list(map(list, it.product([0, 1], repeat=9)))
listString = list(map("".join, it.product('01', repeat=9)))
for row in listInteger:
    row.insert(0, listString[listInteger.index(row)])
matrices = np.array(listInteger)

# 2. display some matrices
rows = 1
columns = 5
fig = plt.figure(figsize=(10,10))
for i in range( rows * columns):
    lblsX = ['A', 'B', 'C']
    lblsY = ['1', '2', '3']
    img = matrices[i, 1:10].astype(np.integer).reshape(3,3)
    ax = fig.add_subplot(rows, columns, i+1)
    #ax.title.set_text(matrices[i,0])
    ax.set_title(matrices[i,0], fontsize=18)
    ax.set_yticks([0,1,2])
    ax.set_xticks([0,1,2])
    ax.set_xticklabels(lblsX, fontsize=18)
    ax.set_yticklabels(lblsY, fontsize=18)
    ax.imshow(img, cmap="Greys" )
plt.show()



In [ ]:
# Save all matrices
# In Paper: 3. Methodology - 3.1.2. Implementation of task characteristics

for matrix in matrices:
    lblsX = ['A', 'B', 'C']
    lblsY = ['1', '2', '3']
    img = matrix[1:10].astype(np.integer).reshape(3,3)
    ax = plt.subplot()
    plt.rcParams.update({'font.size': 56})
    #ax.title.set_text(matrix[0])
    ax.set_yticks([0,1,2])
    ax.set_xticks([0,1,2])
    ax.set_xticklabels(lblsX)
    ax.set_yticklabels(lblsY)
    ax.xaxis.tick_top()
    #fig.savefig('path/to/save/image/to.png')   # save the figure to file
    #plt.close(fig) 
    ax.imshow(img, cmap="Greys" )
    #name = matrix[0] + ".png"
    #print(name)
    print("images/" + matrix[0] + ".png")
    plt.savefig("images/" + matrix[0] + ".png", bbox_inches='tight')
    plt.close(fig) 

# Part 3: Rule

In [ ]:
# Split the set into two nearly equal parts by a rule
# In Paper: 3. Methodology - 3.1.2. Implementation of task characteristics

#edge to edge
#Diagonal Options
selectDiagonalDesc1 = matrices[:, 3].astype(np.integer) == 1
selectDiagonalDesc2 = np.sum(matrices[:, (2,6)].astype(np.integer), axis=1) == 2
selectDiagonalDesc3 = np.sum(matrices[:, (1,5,9)].astype(np.integer), axis=1) == 3
selectDiagonalDesc4 = np.sum(matrices[:, (4,8)].astype(np.integer), axis=1) == 2
selectDiagonalDesc5 = matrices[:, 7].astype(np.integer) == 1

selectDiagonalAsc1 =  matrices[:, 1].astype(np.integer) == 1
selectDiagonalAsc2 =  np.sum(matrices[:, (4,2)].astype(np.integer), axis=1) == 2
selectDiagonalAsc3 =  np.sum(matrices[:, (7,5,3)].astype(np.integer), axis=1) == 3
selectDiagonalAsc4 =  np.sum(matrices[:, (8,6)].astype(np.integer), axis=1) == 2
selectDiagonalAsc5 =  matrices[:, 9].astype(np.integer) == 1

selectDiagonal = selectDiagonalDesc3 | selectDiagonalAsc3

#Vertical Options
selectVertical1 = np.sum(matrices[:, (1,4,7)].astype(np.integer), axis=1) == 3
selectVertical2 = np.sum(matrices[:, (2,5,8)].astype(np.integer), axis=1) == 3
selectVertical3 = np.sum(matrices[:, (3,5,9)].astype(np.integer), axis=1) == 3

#Horizontal Options
selectHorizontal1 = np.sum(matrices[:, (1,2,3)].astype(np.integer), axis=1) == 3
selectHorizontal2 = np.sum(matrices[:, (4,5,6)].astype(np.integer), axis=1) == 3
selectHorizontal3 = np.sum(matrices[:, (7,8,9)].astype(np.integer), axis=1) == 3

selectHorizontal = selectHorizontal1 | selectHorizontal2 | selectHorizontal3

#numbers Options
selectNumbersEquals5 = np.sum(matrices[:, 1:].astype(np.integer), axis=1) == 5
selectNumbersSmaller5 = np.sum(matrices[:, 1:].astype(np.integer), axis=1) < 5
selectNumbersMod2 = np.sum(matrices[:, 1:].astype(np.integer), axis=1) % 2 == 0

selectNumbers = selectNumbersEquals5

#Symmetry Options
selectSymmetryVertical = (np.sum(matrices[:, (1,3)].astype(np.integer), axis=1) % 2 == 0) & (np.sum(matrices[:, (4,6)].astype(np.integer), axis=1) % 2 == 0) & (np.sum(matrices[:, (7,9)].astype(np.integer), axis=1) % 2 == 0)
selectSymmetryHorizontal = (np.sum(matrices[:, (1,7)].astype(np.integer), axis=1) % 2 == 0) & (np.sum(matrices[:, (2,8)].astype(np.integer), axis=1) % 2 == 0) & (np.sum(matrices[:, (3,9)].astype(np.integer), axis=1) % 2 == 0)
selectSymmetryPoint = (np.sum(matrices[:, (1,9)].astype(np.integer), axis=1) % 2 == 0) & (np.sum(matrices[:, (2,8)].astype(np.integer), axis=1) % 2 == 0) & (np.sum(matrices[:, (3,7)].astype(np.integer), axis=1) % 2 == 0) & (np.sum(matrices[:, (4,6)].astype(np.integer), axis=1) % 2 == 0)

selectSymmetry = selectSymmetryHorizontal | selectSymmetryVertical

# Selection
select = selectSymmetry

print(sorted(Counter(select).items()))

# Part 4: Training and Test Set

In [ ]:
# Select training and test set (random_state for testing)
# In Paper: 3. Methodology - 3.2. Experiment design

# undersampling
rus = RandomUnderSampler(random_state=0)
#X_resampled, y_resampled = rus.fit_resample(matrices[:, 1:10].astype(np.integer), select)
X_resampled, y_resampled = rus.fit_resample(matrices, select)


print(sorted(Counter(y_resampled).items()))

# train and test split (test_size = 0.5 because adding 5 and testing on 5)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.5, random_state=0)

# Part 5: Grid Search (optional)

In [ ]:
# Grid search to find optimal parameter configuration
# In Paper: 5. Discussion - 5.2.1 Hyperparameter tuning

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

#Save results in dataframe
df = pd.DataFrame(columns=["user_id", "exp_id", "session", "game_nr", "rule", "batch_no", "time_diff", "count", "count percentage", "AI", "Parameter"] )

t1 = datetime.now()
for j in range (0,42):
    # undersampling
    rus = RandomUnderSampler()
    selectDiagonal = selectDiagonalDesc3 | selectDiagonalAsc3
    selectHorizontal = selectHorizontal1 | selectHorizontal2 | selectHorizontal3
    selectNumbers = selectNumbersEquals5 #selectNumbersMod2
    selectSymmetry = selectSymmetryHorizontal | selectSymmetryVertical
    select = selectSymmetry  
    rule = "selectSymmetry"
    
    X_resampled, y_resampled = rus.fit_resample(matrices, select)
    # 50% Grid, 50% Experiment
    X_grid, X_exp, y_grid, y_exp = train_test_split(X_resampled, y_resampled, test_size=0.5) #, random_state=0)
    # 50% train, 50% test
    X_train, X_test, y_train, y_test = train_test_split(X_exp, y_exp, test_size=0.5)  
     
    #Grid Search
    parameters = {'solver':('adam', 'lbfgs'), 'hidden_layer_sizes':((100),(100,100),(100,100,100)), 'activation': ('relu', 'tanh'), 'learning_rate': ('constant', 'adaptive'), 'learning_rate_init': (0.001, 0.01, 0.1 )} #, 'warm_start':(True, False), }
    mlp = MLPClassifier()
    clf = GridSearchCV(mlp, parameters)
    clf.fit(X_grid[0:99, 1:].astype(np.integer), y_grid[0:99])
    print("Best_params:")
    print(clf.best_params_)
    
    best_parameter = clf.best_params_
    
    
    #Experiment
    rounds = 10       
    for k in range(0, rounds):
        # predict
        name = "MLP"
        mlp_opt = clf.best_estimator_
        print("Round: "+ str(j) + " Batch: "+ str(k))
        print(mlp_opt)
        mlp_opt.fit(X_train[0:(5+5*k), 1:].astype(np.integer), y_train[0:(5+5*k)])
        auc = mlp_opt.score(X_test[(5*k):(5+5*k), 1:].astype(np.integer), y_test[(5*k):(5+5*k)])
        
        t2 = datetime.now()
        delta = t2-t1
        df2 = pd.DataFrame(data=[["AI_"+name+"_Nr_"+str(j)+"_Batch_"+str(k),"AI_"+name+"_Nr_"+str(j)+"_Rule_"+rule+"_Batch_"+str(k),1,0,rule,k,delta.seconds,auc*5,auc,name,best_parameter]], columns=["user_id", "exp_id", "session", "game_nr", "rule", "batch_no", "time_diff", "count", "count percentage", "AI", "Parameter"] )
        df = df.append(df2, ignore_index=True)
   

## save to xlsx file

filepath = 'Grid_Search_MLP_Symmetry.xlsx'
df.to_excel(filepath, index=False)



# Part 6: Step-by-step Predicting (Optional)

In [ ]:
# Show first 5
# In Paper: 3. Methodology - 3.2.2. Experiment with machines

rows = 1
columns = 5
fig = plt.figure(figsize=(10,10))
for i in range( rows * columns):
    img = X_train[i, 1:].astype(np.integer).reshape(3,3)
    ax = fig.add_subplot(rows, columns, i+1)
    ax.set_title(y_train[i], fontsize=18)
    ax.set_yticks([0,1,2])
    ax.set_xticks([0,1,2])
    ax.set_xticklabels(lblsX, fontsize=18)
    ax.set_yticklabels(lblsY, fontsize=18)
    ax.imshow(img, cmap="Greys" )
plt.show()


In [ ]:
# Predict with ML
# In Paper: 3. Methodology - 3.2.2. Experiment with machines

logreg = LogisticRegression(solver='lbfgs')
logreg.fit(X_train[0:5, 1:].astype(np.integer), y_train[0:5]) #.astype(np.integer))
score = logreg.score(X_test[0:5, 1:].astype(np.integer), y_test[0:5]) #.astype(np.integer))

print(score)

In [ ]:
# Show results
# In Paper: 3. Methodology - 3.2.2. Experiment with machines

rows = 1
columns = 5
fig = plt.figure(figsize=(10,10))
for i in range(0,5):
    img = X_test[i, 1:].astype(np.integer).reshape(3,3)
    ax = fig.add_subplot(rows, columns, i+1)  
    s= logreg.predict(X_test[i:(i+1), 1:].astype(np.integer))[0] , '->', y_test[i]  #.astype(np.integer)  
    ax.set_title(s, fontsize=10)
    ax.set_yticks([0,1,2])
    ax.set_xticks([0,1,2])
    ax.set_xticklabels(lblsX, fontsize=18)
    ax.set_yticklabels(lblsY, fontsize=18)
    ax.imshow(img, cmap="Greys" )
plt.show()


In [ ]:
# Continue Predicting
# In Paper: 3. Methodology - 3.2.2. Experiment with machines

# 19 more loops (until 100 samples)
for k in range(1, 20):
    
    # show
    print("Round ", (k+1))
    rows = 1
    columns = 5
    fig = plt.figure(figsize=(10,10))
    for i in range( rows * columns):
        img = X_train[i+k*5, 1:].astype(np.integer).reshape(3,3)
        ax = fig.add_subplot(rows, columns, i+1)
        #ax.title.set_text(y_train[i+k*5])
        ax.set_title(y_train[i+k*5], fontsize=10)
        ax.set_yticks([0,1,2])
        ax.set_xticks([0,1,2])
        ax.set_xticklabels(lblsX, fontsize=18)
        ax.set_yticklabels(lblsY, fontsize=18)
        ax.imshow(img, cmap="Greys" )
    plt.show()

    # predict
    logreg = LogisticRegression(solver='lbfgs')
    logreg.fit(X_train[0:(5+5*k), 1:].astype(np.integer), y_train[0:(5+5*k)])
    logreg.score(X_test[0:(5+5*k), 1:].astype(np.integer), y_test[0:(5+5*k)])
    #print(logit_roc_auc)

    # show
    rows = 1
    columns = 5
    fig = plt.figure(figsize=(10,10))
    for i in range(0,5):
        img = X_test[i+5*k, 1:].astype(np.integer).reshape(3,3)
        ax = fig.add_subplot(rows, columns, i+1)
        s= logreg.predict(X_test[(i+5*k):(i+5*k+1), 1:].astype(np.integer))[0], '->', y_test[i+5*k]
        #ax.title.set_text(X_test[i+5*k, 0])
        ax.set_title(X_test[i+5*k, 0], fontsize=10)
        ax.set_yticks([0,1,2])
        ax.set_xticks([0,1,2])
        ax.set_xticklabels(lblsX, fontsize=18)
        ax.set_yticklabels(lblsY, fontsize=18)
        ax.imshow(img, cmap="Greys" )
    plt.show()


# Part 7: Conduct experiment with machines

In [ ]:
# Experiment with machines (run for each rule)
# In Paper: 4. Results - 4.2. Experiment with supervised machine learning models

#Save results in dataframe
df = pd.DataFrame(columns=["user_id", "exp_id", "session", "game_nr", "rule", "batch_no", "time_diff", "count", "count percentage", "AI"] )

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(20,5)) #, sharey=True)
#ax1.ylim=(0,1)
ax1.set_ylim(0,1)
ax1.title.set_text("Logistic Regression")
ax2.set_ylim(0,1)
ax2.title.set_text("Desicion Tree")
ax3.set_ylim(0,1)
ax3.title.set_text("Neural Network")

t1 = datetime.now()

for j in range (0,42):
    # undersampling
    rus = RandomUnderSampler() #random_state=0)
    selectDiagonal = selectDiagonalDesc3 | selectDiagonalAsc3
    selectHorizontal = selectHorizontal1 | selectHorizontal2 | selectHorizontal3
    selectNumbers = selectNumbersEquals5 #selectNumbersMod2
    selectSymmetry = selectSymmetryHorizontal | selectSymmetryVertical
    
    select = selectDiagonal  #42
    rule = "selectDiagonal"
    #select = selectHorizontal   #42
    #rule = "selectHorizontal"
    #select = selectNumbers  #43
    #rule = "selectNumbers"
    #select = selectSymmetry  #43
    #rule = "selectSymmetry"
    
    X_resampled, y_resampled = rus.fit_resample(matrices, select)
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.5) #, random_state=0)
    
    #ax = fig.add_subplot(1, 2, 1)
    #ax.ylim=(0,1)
    rounds = 20
    for k in range(0, rounds):
        # predict
        logreg = LogisticRegression(solver='lbfgs')
        name = "LogReg"
        try:
            logreg.fit(X_train[0:(5+5*k), 1:].astype(np.integer), y_train[0:(5+5*k)])
            acc = logreg.score(X_test[(5*k):(5+5*k), 1:].astype(np.integer), y_test[(5*k):(5+5*k)])
        except:
            acc = 0.5 #random guess
        #auc = logreg.score(X_test[0:(5+5*k), 1:].astype(np.integer), y_test[0:(5+5*k)])
        ax1.scatter(5+5*k, auc)
        
        t2 = datetime.now()
        delta = t2-t1
        df2 = pd.DataFrame(data=[["AI_"+name+"_Nr_"+str(j)+"_Batch_"+str(k),"AI_"+name+"_Nr_"+str(j)+"_Rule_"+rule+"_Batch_"+str(k),1,0,rule,k,delta.seconds,acc*5,acc,name]], columns=["user_id", "exp_id", "session", "game_nr", "rule", "batch_no", "time_diff", "count", "count percentage", "AI"] )
        df = df.append(df2, ignore_index=True)
        
    for k in range(0, rounds):
        # predict
        name = "DecisionTree"
        clf = tree.DecisionTreeClassifier()
        clf.fit(X_train[0:(5+5*k), 1:].astype(np.integer), y_train[0:(5+5*k)])
        acc = clf.score(X_test[(5*k):(5+5*k), 1:].astype(np.integer), y_test[(5*k):(5+5*k)])
        ax2.scatter(5+5*k, auc)
        
        t2 = datetime.now()
        delta = t2-t1
        df2 = pd.DataFrame(data=[["AI_"+name+"_Nr_"+str(j)+"_Batch_"+str(k),"AI_"+name+"_Nr_"+str(j)+"_Rule_"+rule+"_Batch_"+str(k),1,0,rule,k,delta.seconds,acc*5,acc,name]], columns=["user_id", "exp_id", "session", "game_nr", "rule", "batch_no", "time_diff", "count", "count percentage", "AI"] )
        df = df.append(df2, ignore_index=True)
        
    for k in range(0, rounds):
        # predict
        name = "MLP"
        mlp = MLPClassifier(solver='lbfgs')
        #better performance:
        #mlp = MLPClassifier(activation='tanh', hidden_layer_sizes = (10,10,10,10,), solver='lbfgs') #hidden_layer_sizes=(20,20),max_iter=500
        mlp.fit(X_train[0:(5+5*k), 1:].astype(np.integer), y_train[0:(5+5*k)])
        acc = mlp.score(X_test[(5*k):(5+5*k), 1:].astype(np.integer), y_test[(5*k):(5+5*k)])
        ax3.scatter(5+5*k, auc)
        
        t2 = datetime.now()
        delta = t2-t1
        df2 = pd.DataFrame(data=[["AI_"+name+"_Nr_"+str(j)+"_Batch_"+str(k),"AI_"+name+"_Nr_"+str(j)+"_Rule_"+rule+"_Batch_"+str(k),1,0,rule,k,delta.seconds,acc*5,acc,name]], columns=["user_id", "exp_id", "session", "game_nr", "rule", "batch_no", "time_diff", "count", "count percentage", "AI"] )
        df = df.append(df2, ignore_index=True)

        

print(df)   
plt.show()

# Part 8: Conduct experiment with machines - CNN (Optional)

In [ ]:
# CNN with Tensorflow
# In Paper: 4. Results - 4.2. Experiment with supervised machine learning models
# In Paper: Appendix - CNN Results


#Save results in dataframe
df = pd.DataFrame(columns=["user_id", "exp_id", "session", "game_nr", "rule", "batch_no", "time_diff", "count", "count percentage", "AI"] )

t1 = datetime.now()

for j in range (0,42):
    # undersampling
    rus = RandomUnderSampler() #random_state=0)
    selectDiagonal = selectDiagonalDesc3 | selectDiagonalAsc3
    selectHorizontal = selectHorizontal1 | selectHorizontal2 | selectHorizontal3
    selectNumbers = selectNumbersEquals5 #selectNumbersMod2
    selectSymmetry = selectSymmetryHorizontal | selectSymmetryVertical
    
    select = selectDiagonal  #42
    rule = "selectDiagonal"
    #select = selectHorizontal   #42
    #rule = "selectHorizontal"
    #select = selectNumbers  #43
    #rule = "selectNumbers"
    #select = selectSymmetry  #43
    #rule = "selectSymmetry"
    
    X_resampled, y_resampled = rus.fit_resample(matrices, select)
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.5) #, random_state=0)
    
    rounds = 20
    for k in range(0, rounds):
        # predict
        name = "CNN"
        model = Sequential()
        
        # input layer with 9 neurons, since matrices with 3x3 = 9 elements
        model.add(layers.Dense(units=9, activation="relu")) #, input_shape=(X_train.shape[-1],) ))
        
        # randomly delete 30% of the input units below
        model.add(layers.Dropout(0.3))
        model.add(layers.Dense(units=9, activation="relu"))
        
        # the output layer, with a single neuron
        model.add(layers.Dense(units=1, activation="sigmoid"))

        model.compile(
            optimizer = 'adam',
            loss = 'binary_crossentropy',
            metrics = ['accuracy']
        )
        model.fit(X_train[0:(5+5*k), 1:].astype(np.integer), y_train[0:(5+5*k)]) 
        loss, acc = model.evaluate(X_test[(5*k):(5+5*k), 1:].astype(np.integer), y_test[(5*k):(5+5*k)])
           
        t2 = datetime.now()
        delta = t2-t1
        df2 = pd.DataFrame(data=[["AI_"+name+"_Nr_"+str(j)+"_Batch_"+str(k),"AI_"+name+"_Nr_"+str(j)+"_Rule_"+rule+"_Batch_"+str(k),1,0,rule,k,delta.seconds,acc*5,acc,name]], columns=["user_id", "exp_id", "session", "game_nr", "rule", "batch_no", "time_diff", "count", "count percentage", "AI"] )
        df = df.append(df2, ignore_index=True)
        

print(df)   


# Part 9: Confusion matrix (Optional)

In [ ]:
# Confusion matrix
# In Paper: 4. Results - 4.3. Comparison
# In Paper: Appendix - Confusion Matrices

#Save results in dataframe
df = pd.DataFrame(columns=["user_id", "exp_id", "session", "game_nr", "rule", "batch_no", "time_diff", "count", "count percentage", "AI"] )

t1 = datetime.now()

for j in range (0,1):
    # undersampling
    rus = RandomUnderSampler() #random_state=0)
    selectDiagonal = selectDiagonalDesc3 | selectDiagonalAsc3
    selectHorizontal = selectHorizontal1 | selectHorizontal2 | selectHorizontal3
    selectNumbers = selectNumbersEquals5 #selectNumbersMod2
    selectSymmetry = selectSymmetryHorizontal | selectSymmetryVertical
    
    select = selectDiagonal  #42
    rule = "selectDiagonal"
    #select = selectHorizontal   #42
    #rule = "selectHorizontal"
    #select = selectNumbers  #43
    #rule = "selectNumbers"
    #select = selectSymmetry  #43
    #rule = "selectSymmetry"
    
    X_resampled, y_resampled = rus.fit_resample(matrices, select)
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.5) # random_state=0)
    
    rounds = 10
    
    for k in range(9, rounds):
        # predict
        name = "DecisionTree"
        clf = tree.DecisionTreeClassifier()
        clf.fit(X_train[0:(5+5*k), 1:].astype(np.integer), y_train[0:(5+5*k)])
        acc = clf.score(X_test[(5*k):(5+5*k), 1:].astype(np.integer), y_test[(5*k):(5+5*k)])
        tn, fp, fn, tp = confusion_matrix(y_test[0:(5+5*k)], mlp.predict(X_test[0:(5+5*k), 1:].astype(np.integer))).ravel()
        
        name = "LogReg"
        logreg = LogisticRegression(solver='lbfgs')
        logreg.fit(X_train[0:(5+5*k), 1:].astype(np.integer), y_train[0:(5+5*k)])
        acc = logreg.score(X_test[(5*k):(5+5*k), 1:].astype(np.integer), y_test[(5*k):(5+5*k)])
        tn, fp, fn, tp = confusion_matrix(y_test[0:(5+5*k)], logreg.predict(X_test[0:(5+5*k), 1:].astype(np.integer))).ravel()
        




# Part 10: Save Results

In [ ]:
# Save to xlsx file

filepath = 'my_excel_file.xlsx'

df.to_excel(filepath, index=False)